In [36]:
using JuMP
using Gurobi

## Monolith

In [37]:
using JuMP
using Gurobi

m = Model(solver = GurobiSolver())
@variable(m,x>=0)
@variable(m,y>=0)
@variable(m,z>=0)
@variable(m,t>=0)

@constraint(m, x+y>=60)
@constraint(m, y+z>=80)
@constraint(m, x+t>=43)

@objective(m,Min,x+ 3y + 5z + 0.5t)
print(m)
solve(m)
println("x=", getvalue(x))
println("y=", getvalue(y))
println("z=", getvalue(z))
println("t=", getvalue(t))
println("obj = ", getobjectivevalue(m))

Min x + 3 y + 5 z + 0.5 t
Subject to
 x + y ≥ 60
 y + z ≥ 80
 x + t ≥ 43
 x ≥ 0
 y ≥ 0
 z ≥ 0
 t ≥ 0
Optimize a model with 3 rows, 4 columns and 6 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 8e+01]
Presolve removed 3 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.6150000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  2.615000000e+02
x=0.0
y=80.0
z=0.0
t=43.0
obj = 261.5


In [38]:
#A = JuMP.prepConstrMatrix(m)
#A = full(A)
#spy(A)

## Benders

In [39]:
i = [:i1, :i2, :i3, :i4]

λᵏ = Dict(:i2 => 0, :i3 => 0, :i4 => 0)
μᵏ = Dict(:i1 => 0, :i2 => 0, :i3 => 0)
θᵏ = Dict(:i1 => 0, :i2 => 0, :i3 => 0, :i4 => 0)
zbᵏ = Dict(:i1 => 0, :i2 => 0, :i3 => 0, :i4 => 0)
zlᵏ = Dict(:i1 => 0, :i2 => 0, :i3 => 0, :i4 => 0)

duals = []

mp = Model(solver = GurobiSolver())
@variable(mp,80 >= x >=0)
@variable(mp,80 >= xᵏ>=0)
@variable(mp,θ[1:3])
@objective(mp,Min, θ[1])
@constraint(mp, θ[1] >= x + θ[2] + θ[3])

function refreshmp()
    x = getindex(mp,:x)
    θ = getindex(mp,:θ)
    xᵏ = getindex(mp, :xᵏ)
    @constraint(mp, θ[2] >= θᵏ[:i2] + λᵏ[:i2]*(xᵏ - x))
    @constraint(mp, θ[3] >= θᵏ[:i3] + λᵏ[:i3]*(xᵏ - x))
    @constraint(mp, zlᵏ[:i1] <= θ[1] + μᵏ[:i1]*x + μᵏ[:i3]*x)
    print(mp)
end
refreshmp()

Min θ[1]
Subject to
 θ[1] - x - θ[2] - θ[3] ≥ 0
 θ[2] ≥ 0
 θ[3] ≥ 0
 -θ[1] ≤ 0
 θ[i] ∀ i ∈ {1,2,3}
 0 ≤ x ≤ 80
 0 ≤ xᵏ ≤ 80


In [40]:
sp1 = Model(solver = GurobiSolver())
@variable(sp1,80 >= x>=0)
@variable(sp1,80 >= y>=0)
@variable(sp1,80 >= yᵏ>=0)
@variable(sp1,θ[2:4])
@variable(sp1,80 >= xᵏ>=0)
@constraint(sp1, x+y>=60)
dual2 = @constraint(sp1, xᵏ - x == 0)
push!(duals, dual2)
@objective(sp1, Min, θ[2])
@constraint(sp1, θ[2] >= 3y + θ[4])

function refreshsp1()
    x = getindex(sp1,:x)
    y = getindex(sp1,:y)
    θ = getindex(sp1,:θ)
    yᵏ = getindex(sp1, :yᵏ)
    @constraint(sp1, θ[4] >= θᵏ[:i4] + λᵏ[:i4]*(yᵏ - y))
    @constraint(sp1, zlᵏ[:i2] <= θ[2] - μᵏ[:i1]*x + μᵏ[:i2]*y)
    
    print(sp1)
end
refreshsp1()

Min θ[2]
Subject to
 x + y ≥ 60
 xᵏ - x = 0
 θ[2] - 3 y - θ[4] ≥ 0
 θ[4] ≥ 0
 -θ[2] ≤ 0
 θ[i] ∀ i ∈ {2,3,4}
 0 ≤ x ≤ 80
 0 ≤ y ≤ 80
 0 ≤ yᵏ ≤ 80
 0 ≤ xᵏ ≤ 80


In [41]:
 sp3 = Model(solver = GurobiSolver())
@variable(sp3,80 >= x>=0)
@variable(sp3,80 >= xᵏ>=0)
@variable(sp3,80 >= t>=0)
@variable(sp3,θ[3:3])
@constraint(sp3,x+t>=43)
dual3 = @constraint(sp3, xᵏ - x == 0)
push!(duals, dual3)
@constraint(sp3, θ[3] >= 0.5t)
@objective(sp3,Min,θ[3])
function refreshsp3()
    x = getindex(sp3,:x)
    θ = getindex(sp3,:θ)
    @constraint(sp3, zlᵏ[:i3] <= θ[3] - μᵏ[:i3]*x)
    print(sp3)
end
refreshsp3()

Min θ[3]
Subject to
 x + t ≥ 43
 xᵏ - x = 0
 θ[3] - 0.5 t ≥ 0
 -θ[3] ≤ 0
 θ[i] ∀ i ∈ {3}
 0 ≤ x ≤ 80
 0 ≤ xᵏ ≤ 80
 0 ≤ t ≤ 80


In [42]:
sp2 = Model(solver=GurobiSolver())
@variable(sp2,80 >= y>=0)
@variable(sp2,80 >= z>=0)
@variable(sp2,80 >= yᵏ>=0)
@variable(sp2,θ[4:4])
@constraint(sp2, y+z>=80)
dual4 = @constraint(sp2, yᵏ - y == 0)
@constraint(sp2, θ[4] >= 5z)
@objective(sp2, Min, θ[4])
function refreshsp2()
    y = getindex(sp2,:y)
    θ = getindex(sp2,:θ)
    @constraint(sp2, zlᵏ[:i4] <= θ[4] - μᵏ[:i2]*y)
    print(sp2)
end
refreshsp2()

Min θ[4]
Subject to
 y + z ≥ 80
 yᵏ - y = 0
 θ[4] - 5 z ≥ 0
 -θ[4] ≤ 0
 θ[i] ∀ i ∈ {4}
 0 ≤ y ≤ 80
 0 ≤ z ≤ 80
 0 ≤ yᵏ ≤ 80


## Lagrange

### Master
$ \max \> \eta \\
s.t. \> \> \eta \leq \sum\kappa_s\\
$


In [43]:
xlᵏ = 0
ylᵏ = 0
xbᵏ = 0
ybᵏ = 0

LMP = Model(solver = GurobiSolver())
@variable(LMP, κ[1:4])
@variable(LMP, η)
@variable(LMP, μ[1:3])
@constraint(LMP, η <= sum(κ[i] for i in 1:4))
@objective(LMP, Max, η)

function refreshlmp()

    @constraint(LMP, κ[1] <= zbᵏ[:i1] + μ[1]*xbᵏ + μ[3]*xbᵏ)
    @constraint(LMP, κ[2] <= zbᵏ[:i2] - μ[1]*xbᵏ + μ[2]*ybᵏ )
    @constraint(LMP, κ[3] <= zbᵏ[:i3] - μ[3]*xbᵏ )
    @constraint(LMP, κ[4] <= zbᵏ[:i4] - μ[2]*ybᵏ)
    @constraint(LMP, κ[1] <= zlᵏ[:i1] + μ[1]*xlᵏ + μ[3]*xlᵏ)
    @constraint(LMP, κ[2] <= zlᵏ[:i2] - μ[1]*xlᵏ + μ[2]*ylᵏ )
    @constraint(LMP, κ[3] <= zlᵏ[:i3] - μ[3]*xlᵏ )
    @constraint(LMP, κ[4] <= zlᵏ[:i4] - μ[2]*ylᵏ)
    

    print(LMP)
end
refreshlmp()

Max η
Subject to
 η - κ[1] - κ[2] - κ[3] - κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[i] ∀ i ∈ {1,2,3,4}
 μ[i] ∀ i ∈ {1,2,3}
 η


In [44]:
lsp1 = Model(solver=GurobiSolver())
@variable(lsp1,80 >= x>=0)
function refreshlsp1()
    x = getindex(lsp1,:x)
    @objective(lsp1,Min,x+ μᵏ[:i1]*x  + μᵏ[:i3]*x )
    print(lsp1)
end
refreshlsp1()

Min x
Subject to
 0 ≤ x ≤ 80


In [45]:
lsp2 = Model(solver=GurobiSolver())
@variable(lsp2,80 >= x>=0)
@variable(lsp2,80 >= y>=0)
@constraint(lsp2, x+y>=60)
function refreshlsp2()
    x = getindex(lsp2,:x)
    y = getindex(lsp2,:y)
    @objective(lsp2, Min, 3y - μᵏ[:i1]*x + μᵏ[:i2]*y)
    print(lsp2)
end
refreshlsp2()

Min 3 y
Subject to
 x + y ≥ 60
 0 ≤ x ≤ 80
 0 ≤ y ≤ 80


In [46]:
lsp3 = Model(solver=GurobiSolver())
@variable(lsp3,80 >= x>=0)
@variable(lsp3,80 >= t>=0)
@constraint(lsp3,x+t>=43)
function refreshlsp3()
    x = getindex(lsp3,:x)
    t = getindex(lsp3,:t)
    @objective(lsp3,Min,.5t- μᵏ[:i3]*x)
    print(lsp3)
end
refreshlsp3()

Min 0.5 t
Subject to
 x + t ≥ 43
 0 ≤ x ≤ 80
 0 ≤ t ≤ 80


In [47]:
lsp4 = Model(solver=GurobiSolver())
@variable(lsp4,80 >= y>=0)
@variable(lsp4,80 >= z>=0)
@constraint(lsp4, y+z>=80)
function refreshlsp4()
    y = getindex(lsp4,:y)
    z = getindex(lsp4,:z)
    @objective(lsp4, Min, 5z - μᵏ[:i2]*y)
    print(lsp4)
end
refreshlsp4()

Min 5 z
Subject to
 y + z ≥ 80
 0 ≤ y ≤ 80
 0 ≤ z ≤ 80


In [48]:
function refreshall()
    refreshlmp()
    refreshlsp1()
    refreshlsp2()
    refreshlsp3()
    refreshlsp4()
    refreshmp()
    refreshsp1()
    refreshsp2()
    refreshsp3()
end

refreshall (generic function with 1 method)

Moving through algorithm:

In [49]:
#Initialize

for iter in 1:50
#Benders Forward Step
println("h1")
solve(mp)
xbᵏ = getvalue(getindex(mp,:x))
JuMP.fix(getindex(sp1,:xᵏ),float(xbᵏ))
JuMP.fix(getindex(sp3,:xᵏ),float(xbᵏ))
solve(sp1)
solve(sp3)
ybᵏ = getvalue(getindex(sp1,:y))
JuMP.fix(getindex(sp2,:yᵏ),float(ybᵏ))
solve(sp2)
println("h2")

θᵏ = Dict(:i1 => getobjectivevalue(mp), :i2 => getobjectivevalue(sp1), :i3 => getobjectivevalue(sp3), :i4 => getobjectivevalue(sp2))
zbᵏ = Dict(:i1 => getobjectivevalue(mp), :i2 => getobjectivevalue(sp1), :i3 => getobjectivevalue(sp3), :i4 => getobjectivevalue(sp2))



#Benders Backward Step
λᵏ[:i4] = getdual(dual4)
println("h3")

refreshsp1()
println("h6")
solve(sp1)
λᵏ[:i2] = getdual(dual2)
#λᵏ[:i3] = getdual(dual3)
println("h5")

refreshall()

println("h7")
#=
#Lagrange Forward Step
xbᵏ = getvalue(getindex(mp,:x))
ybᵏ = getvalue(getindex(sp1,:y))
solve(LMP)
μ = getindex(LMP, :μ)
μᵏ = Dict(:i1 => getvalue(μ[1]), :i2 => getvalue(μ[2]), :i3 => getvalue(μ[3]))

println("h8")

refreshall()

println("h9")

solve(lsp1)
println("h10")
solve(lsp2)
println("h11")
solve(lsp3)
println("h12")
print(lsp4)
solve(lsp4)
println("h13")

#Lagrange Backward Step
zlᵏ = Dict(:i1 => getobjectivevalue(lsp1), :i2 => getobjectivevalue(lsp2), :i3 => getobjectivevalue(lsp3), :i4 => getobjectivevalue(lsp4))
println("h14")
refreshall()
println("h15")
=#
end


h1
Optimize a model with 4 rows, 5 columns and 7 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e+01, 8e+01]
  RHS range        [0e+00, 0e+00]
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  0.000000000e+00
Optimize a model with 5 rows, 7 columns and 9 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e+01, 8e+01]
  RHS range        [6e+01, 6e+01]
Presolve removed 5 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+02   0.000000e+00   2.000000e-06      0s

Solved in 0 iterations and 0.00 second

h3
Min θ[2]
Subject to
 x + y ≥ 60
 xᵏ - x = 0
 θ[2] - 3 y - θ[4] ≥ 0
 θ[4] ≥ 0
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 300
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 300
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[i] ∀ i ∈ {2,3,4}
 0 ≤ x ≤ 80
 0 ≤ y ≤ 80
 0 ≤ yᵏ ≤ 80
 xᵏ = 40
h6
Optimize a model with 15 rows, 7 columns and 29 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e+01, 8e+01]
  RHS range        [6e+01, 3e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.800000000e+02
h5
Max η
Subject to
 η - κ[1] - κ[2] - κ[3] - κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] - 60 μ[2] ≤ 180
 κ[3] ≤ 21.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤

h5
Max η
Subject to
 η - κ[1] - κ[2] - κ[3] - κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] - 60 μ[2] ≤ 180
 κ[3] ≤ 21.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] - 60 μ[1] - 60 μ[3] ≤ 81.5
 κ[2] + 60 μ[1] - 20 μ[2] ≤ 60
 κ[3] + 60 μ[3] ≤ 0
 κ[4] + 20 μ[2] ≤ 300
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] - 40 μ[1] - 40 μ[3] ≤ 121.5
 κ[2] + 40 μ[1] - 60 μ[2] ≤ 180
 κ[3] + 40 μ[3] ≤ 1.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 201.5
 κ[2] - 60 μ[2] ≤ 180
 κ[3] ≤ 21.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 201.5
 κ[2] - 60 μ[2] ≤ 180
 κ[3] ≤ 21.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[i] ∀ i ∈ {1,2,3,4}
 μ[i] ∀ i ∈ {1,2,3}
 η
Min x
Subject to
 0 ≤ x ≤ 80
Min 3 y
Subject to
 x + y ≥ 60
 0 ≤ x ≤ 80
 0 ≤ y ≤ 80
Min 0.5 t
Subject to
 x + t ≥ 43
 0 ≤ x ≤ 80
 0 ≤ t ≤ 80
Min 5 z
Subject to
 y + z ≥ 80
 0 ≤ y ≤ 80
 0 ≤ z ≤ 80

h6
Optimize a model with 31 rows, 7 columns and 61 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e+01, 8e+01]
  RHS range        [6e+01, 3e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.800000000e+02
h5
Max η
Subject to
 η - κ[1] - κ[2] - κ[3] - κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] - 60 μ[2] ≤ 180
 κ[3] ≤ 21.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] - 60 μ[1] - 60 μ[3] ≤ 81.5
 κ[2] + 60 μ[1] - 20 μ[2] ≤ 60
 κ[3] + 60 μ[3] ≤ 0
 κ[4] + 20 μ[2] ≤ 300
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] - 40 μ[1] - 40 μ[3] ≤ 121.5
 κ[2] + 40 μ[1] - 60 μ[2] ≤ 180
 κ[3] + 40 μ[3] ≤ 1.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 201.5
 κ[2] - 60 μ[2] ≤ 180
 κ

h3
Min θ[2]
Subject to
 x + y ≥ 60
 xᵏ - x = 0
 θ[2] - 3 y - θ[4] ≥ 0
 θ[4] ≥ 0
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 300
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 300
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[4] - 5 yᵏ + 5 y ≥ 100
 -θ[2] ≤ 0
 θ[i] ∀ i ∈ {2,3,4}
 0 ≤ x ≤ 80
 0 ≤ y ≤ 80
 0 ≤ yᵏ ≤ 80
 xᵏ = 0
h6
Optimize a model with 39 rows, 7 columns and 77 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e+01, 8e+01]
  RHS range        [6e+01, 3e+02]
Iteration   

...

Min θ[4]
Subject to
 y + z ≥ 80
 yᵏ - y = 0
 θ[4] - 5 z ≥ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 θ[i] ∀ i ∈ {4}
 0 ≤ y ≤ 80
 0 ≤ z ≤ 80
 yᵏ = 60
Min θ[3]
Subject to
 x + t ≥ 43
 xᵏ - x = 0
 θ[3] - 0.5 t ≥ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[

Min x
Subject to
 0 ≤ x ≤ 80
Min 3 y
Subject to
 x + y ≥ 60
 0 ≤ x ≤ 80
 0 ≤ y ≤ 80
Min 0.5 t
Subject to
 x + t ≥ 43
 0 ≤ x ≤ 80
 0 ≤ t ≤ 80
Min 5 z
Subject to
 y + z ≥ 80
 0 ≤ y ≤ 80
 0 ≤ z ≤ 80
Min θ[1]
Subject to
 θ[1] - x - θ[2] - θ[3] ≥ 0
 θ[2] ≥ 0
 θ[3] ≥ 0
 -θ[1] ≤ 0
 θ[2] - 3 xᵏ + 3 x ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 60
 θ[3] ≥ 0
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 1.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤ 0
 θ[2] ≥ 180
 θ[3] ≥ 21.5
 -θ[1] ≤

h6
Optimize a model with 175 rows, 7 columns and 349 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [8e+01, 8e+01]
  RHS range        [6e+01, 3e+02]
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s

Solved in 0 iterations and 0.00 seconds
Optimal objective  1.800000000e+02
h5
Max η
Subject to
 η - κ[1] - κ[2] - κ[3] - κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 0
 κ[2] - 60 μ[2] ≤ 180
 κ[3] ≤ 21.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] - 60 μ[1] - 60 μ[3] ≤ 81.5
 κ[2] + 60 μ[1] - 20 μ[2] ≤ 60
 κ[3] + 60 μ[3] ≤ 0
 κ[4] + 20 μ[2] ≤ 300
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] - 40 μ[1] - 40 μ[3] ≤ 121.5
 κ[2] + 40 μ[1] - 60 μ[2] ≤ 180
 κ[3] + 40 μ[3] ≤ 1.5
 κ[4] + 60 μ[2] ≤ 100
 κ[1] ≤ 0
 κ[2] ≤ 0
 κ[3] ≤ 0
 κ[4] ≤ 0
 κ[1] ≤ 201.5
 κ[2] - 60 μ[2] ≤ 180


Excessive output truncated after 526490 bytes.

Min θ[4]
Subject to
 y + z ≥ 80
 yᵏ - y = 0
 θ[4] - 5 z ≥ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 -θ[4] ≤ 0
 θ[i] ∀ i ∈ {4}
 0 ≤ y ≤ 80
 0 ≤ z ≤ 80
 yᵏ = 60
Min θ[3]
Subject to
 x + t ≥ 43
 xᵏ - x = 0
 θ[3] - 0.5 t ≥ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[3] ≤ 0
 -θ[

In [52]:
getobjectivevalue(mp)

201.5

In [51]:
get

get (generic function with 32 methods)